In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

parent_env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=parent_env_path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
API_KEY = os.getenv("API_KEY_1")
project_id = os.getenv("PROJECT_ID")

In [ ]:
#connect to BigQuery
from google.cloud import bigquery
from google.cloud import storage
from googleapiclient.discovery import build
import json
from datetime import datetime
import pandas as pd
import time
import StringIO
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

In [3]:
#connect to prj_id, dataset_id = 2_cleaned_data, table_id: video_basic_info
client = bigquery.Client(project=project_id)
dataset_id = "2_cleaned_data"
table_id = "video_basic_info"
table_ref = client.dataset(dataset_id).table(table_id)

c:\Users\kt061\anaconda3\envs\min_ds\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [15]:
#querry and get the channel_id
query = f"""
(
  SELECT DISTINCT id
  FROM `ytbdataanalyst.2_cleaned_data.video_basic_info`
)
EXCEPT DISTINCT
(
  SELECT DISTINCT id
  FROM `ytbdataanalyst.2_cleaned_data.video_transcript`
)
"""

In [16]:
# Thực thi truy vấn
query_job = client.query(query)
results = query_job.result()

# Lưu kết quả vào danh sách
video_list = [row.id for row in results]
len(video_list)

1403

In [17]:
video_list = video_list[:500]  # Giới hạn danh sách video để tránh quá tải

In [18]:
def get_transcript_flexible(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        # Ưu tiên phụ đề tiếng Anh
        try:
            return transcript_list.find_transcript(['en']).fetch(), 'en'
        except:
            pass

        # Nếu không có tiếng Anh, lấy bất kỳ cái nào fetch được
        for transcript in transcript_list:
            try:
                return transcript.fetch(), transcript.language_code
            except:
                continue

    except Exception as e:
        print(f"Không lấy được transcript cho video {video_id}: {e}")
        return None,None

In [19]:
# 0 to 4000
data = []
start = 0
end = 500
count = start
for v in video_list[start:end]:
    only_text = ''
    transcript,lang = get_transcript_flexible(v)

    if transcript is None:
        print(f"No transcript found for video {v}")
        data.append({'id': v, 'transcript': 'No Transcript', 'lang': 'N/A'})
        count += 1
        continue

    for item in transcript:
        # only_text += item['text'] + ' '
        only_text += item.text + ' '
    
    data.append({'id': v, 'transcript': only_text, 'lang': lang})
    print(f"Video {count}/{len(video_list)}: {v}")
    count += 1

    time.sleep(3)        
    if count % 100 == 0:
        time.sleep(60)  # sleep for 1 minute every 100 videos to avoid hitting API limits

Video 0/500: 5HYnlA8c8LA
Video 1/500: 7l0G_awxFg0
Video 2/500: LDOFrBnaS0o
Không lấy được transcript cho video 7jVKwYQdKww: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=7jVKwYQdKww! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
No transcript found for video 7jVKwYQdKww
Không lấy được transcript cho video d6-clVn269o: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=d6-clVn269o! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not r

In [22]:
#convert to dataframe
df = pd.DataFrame(data)

In [23]:
df

,id,transcript,lang
0,5HYnlA8c8LA,అందరికీ నమస్కారం వెల్కమ్ బ్యాక్ టు వంశీ భవాని ...,te
1,7l0G_awxFg0,అందరికీ నమస్కారం వెల్కమ్ బ్యాక్ టు వంశీ భవాని ...,te
2,LDOFrBnaS0o,అందరికీ నమస్కారం వెల్కమ్ బ్యాక్ టు వంశీ భవాని ...,te
3,7jVKwYQdKww,No Transcript,N/A
4,d6-clVn269o,No Transcript,N/A
...,...,...,...
495,nrvo5TKns7c,o loucodi está revolucionando o desenvolviment...,pt
496,CInXqRYfP5M,Ela chega e diz,pt
497,c3RAXlYMy9s,No Transcript,N/A
498,y0aFor4ety4,No Transcript,N/A


In [ ]:
#connect and save script to bigquery
# Load config
dataset_id = "2_cleaned_data"
table_id = 'video_transcript'
table_ref = f"{project_id}.{dataset_id}.{table_id}"
full_dataset_id = f"{project_id}.{dataset_id}"
client = bigquery.Client(project=project_id)
dataset = bigquery.Dataset(full_dataset_id)

c:\Users\kt061\anaconda3\envs\min_ds\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,  # Use WRITE_TRUNCATE to overwrite
    autodetect=True  # Let BigQuery detect schema from DataFrame
)

job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

print(f"✅ Data uploaded to BigQuery table: {table_id}")

c:\Users\kt061\anaconda3\envs\min_ds\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Data uploaded to BigQuery table: video_transcript


In [ ]:
# bucket_name = "ytbdata"
# gcs_folder = "1_crawl"
# date_str = datetime.today().strftime("%Y%m%d")
# filename = f"video_transcript_{date_str}.json"
# blob_path = f"{gcs_folder}/video_transcript/{filename}"

In [ ]:
# # --- Chuyển DataFrame thành CSV (nội dung dạng chuỗi) ---
# csv_buffer = StringIO()
# df.to_csv(csv_buffer, index=False, encoding='utf-8-sig')  # utf-8-sig giúp mở trong Excel không lỗi font

# client = storage.Client()
# bucket = client.bucket(bucket_name)
# blob = bucket.blob(blob_path)

# blob.upload_from_string(csv_buffer.getvalue(), content_type="text/csv")
# print(f"✅ Uploaded CSV to: gs://{bucket_name}/{blob_path}")